In [2]:
import os
import re
import pandas as pd
from dotenv import load_dotenv
import time
import asyncio

In [11]:
from utils.vector_store import VectorStore
vector_store_tfidf = VectorStore()

In [12]:
from utils.vector_store_transformers import TransformerVectorStore
vector_store_transformer = TransformerVectorStore()

In [13]:
from utils.pdf_processor import extract_text_from_pdf, chunk_text

UPLOAD_FOLDER = 'uploads'
def load_existing_documents(vector_store):
    if not os.path.exists(UPLOAD_FOLDER):
        return
    
    # Clear existing vector store to avoid duplicate entries
    vector_store.clear()
    global file_information
    file_information = {}  # Reset file information dictionary
    
    pdf_files = [f for f in os.listdir(UPLOAD_FOLDER) if f.endswith('.pdf')]
    
    for pdf_file in pdf_files:
        filepath = os.path.join(UPLOAD_FOLDER, pdf_file)
        try:
            # Extract text from PDF
            text = extract_text_from_pdf(filepath)
            
            # Store file information for reference
            file_id = pdf_file
            file_information[file_id] = {
                'filename': pdf_file,
                'content': text
            }
            
            # Chunk text for vector database and tag with source file
            chunks = chunk_text(text, file_source=pdf_file)
            
            if chunks:
                # Get sample text (skip the source tag line for logging)
                sample_chunk = chunks[0]
                if sample_chunk.startswith("SOURCE_FILE:"):
                    # Extract content after the source tag
                    sample_lines = sample_chunk.split('\n', 1)
                    if len(sample_lines) > 1:
                        sample_chunk = sample_lines[1]
                
                print(f"Sample chunk from {pdf_file}: {sample_chunk[:200]}...")
                print(f"Extracted {len(chunks)} chunks from {pdf_file}")
            
            # Add chunks to vector store with source information
            vector_store.add_documents(chunks, file_source=pdf_file)
            
            print(f"Processed and loaded {pdf_file} into vector store")
        except Exception as e:
           print(f"Error processing PDF {pdf_file}: {e}")

In [14]:
# load_existing_documents(vector_store_tfidf)
load_existing_documents(vector_store_transformer)

Sample chunk from co_so_vat_chat.pdf: THÔNGTINVỀCƠSỞHỌCTẬPCỦATRƯỜNG
I. Cơsởhọctập
Cơsở1: 97VõVănTầnP6Q3Tp. HCM( CơsởVõVănTần-kýhiệuVVT)
-ĐàotạocácngànhChấtlượngcaocủakhoa Đàotạođặcbiệt
Cơsở2: 35-37HồHảoHớn, PhườngCôGiang, Quận1, Tp. HồChí...
Extracted 1 chunks from co_so_vat_chat.pdf
Processed and loaded co_so_vat_chat.pdf into vector store
Sample chunk from diem_chuan.pdf: ĐIỂMCHUẨNQUATỪNGNĂM
Trongđàotạochínhquy, điểmchuẩncủacácngànhtại ĐạihọcMởTP. HCMquacác
nămđượctổnghợpnhưsau:
PhươngthứcxéttuyểnkỳthitốtnghiệpTHPT:
Ngành Quảntrịkinhdoanh:
Năm 2024: 20. 75
Năm 2023: 24...
Extracted 1 chunks from diem_chuan.pdf
Processed and loaded diem_chuan.pdf into vector store
Sample chunk from hoc_phi_hoc_bong.pdf: HỌCPHÍDỰKIẾNCỦACHƯƠNGTRÌNHĐÀOTẠO
ĐHCHÍNHQUYÁPDỤNGKHÓA2025-NĂMHỌC2025-2026
I. Họcphí
Chươngtrìnhchuẩn
1. NhómngànhCôngnghệsinhhọc, Côngnghệthựcphẩm:
-Họcphídựkiến: 28. 500. 000đ/năm
2. NhómngànhCôngngh...
Extracted 1 chunks from hoc_phi_hoc_bong.pdf
Processed and loaded hoc_phi_hoc_bong

In [3]:
df = pd.read_csv('eva_data/test.csv', encoding='utf-8')
df.head()

,query,expected_response
0,Mã trường là gì?,Trường Đại học Mở TP.Hồ Chí Minh có mã là MBS.
1,Sinh viên ngành Công nghệ thông tin học ở cơ s...,Ngành Công nghệ thông tin được đào tạo tại cơ ...
2,Ngành Công nghệ sinh học của trường Đại học Mở...,Ngành Công nghệ sinh học được đào tạo ở Cơ sở ...
3,Điểm chuẩn ngành Ngôn ngữ Nhật Chất lượng cao ...,Điểm chuẩn ngành Ngôn ngữ Nhật Chất lượng cao ...
4,Điểm chuẩn ngành Ngôn ngữ Hàn Quốc theo phương...,Điểm chuẩn ngành Ngôn ngữ Hàn Quốc theo phương...


In [9]:
from bs4 import BeautifulSoup
import html

def clean_html_response(text):
    if not text:
        return ""
    
    # Ghi log nội dung gốc để debug
    debug_sample = text[:100] + "..." if len(text) > 100 else text
    print(f"Nội dung gốc: {debug_sample}")
    
    # Xử lý các mã HTML đặc biệt
    text = html.unescape(text)
    
    # Xử lý các thẻ HTML đặc biệt được bọc trong ```html ... ```
    text = re.sub(r'```html\s*(.*?)\s*```', r'\1', text, flags=re.DOTALL)
    
    # Xử lý các thẻ HTML đặc biệt được bọc trong các dạng markdown khác
    text = re.sub(r'```\s*(.*?)\s*```', r'\1', text, flags=re.DOTALL)
    
    try:
        # Sử dụng BeautifulSoup để loại bỏ thẻ HTML
        soup = BeautifulSoup(text, 'html.parser')
        clean_text = soup.get_text(separator=' ', strip=True)
    except Exception as e:
        print(f"Lỗi khi sử dụng BeautifulSoup: {e}")
        # Fallback: dùng regex nếu BeautifulSoup thất bại
        clean_text = re.sub(r'<[^>]*>', ' ', text)
    
    # Loại bỏ khoảng trắng dư thừa
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()
    
    # Loại bỏ các phần còn sót lại có thể chứa HTML
    clean_text = re.sub(r'<[^>]*>', '', clean_text)
    
    # Xử lý các entity HTML còn sót (như &nbsp;, &gt;, &lt;, etc.)
    clean_text = re.sub(r'&[a-zA-Z]+;', ' ', clean_text)
    
    # Loại bỏ các dòng rỗng
    clean_text = re.sub(r'\n\s*\n', '\n', clean_text)
    
    # Kiểm tra xem còn thẻ HTML nào không
    if re.search(r'<[^>]+>', clean_text):
        print(f"Cảnh báo: Vẫn còn thẻ HTML trong text_response: {clean_text[:100]}...")
        # Cố gắng loại bỏ mọi thứ trông giống HTML
        clean_text = re.sub(r'<[^>]*>', '', clean_text)
        clean_text = re.sub(r'<.*', '', clean_text)
    
    # Thêm kiểm tra final
    if re.search(r'</?[a-z]+[^>]*>', clean_text, re.IGNORECASE):
        print("Vẫn còn HTML sau nhiều lần xử lý, xóa tất cả các ký tự nghi ngờ")
        clean_text = re.sub(r'</?[a-z]+[^>]*>', '', clean_text, flags=re.IGNORECASE)
    
    # Loại bỏ các khoảng trắng dư thừa sau tất cả quá trình xử lý
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()
    
    print(f"Sau khi làm sạch: {clean_text[:100]}...")
    return clean_text

In [25]:
from utils.orchestrator import orchestrate_response
from utils.gemini_api import generate_response

async def generate_responses(vector_store, user_query): 
    start_time = time.time()
    if vector_store.documents:
        # Get more relevant documents (up to 10) with a very low similarity threshold
        context_docs = vector_store.similarity_search(user_query, k=10, threshold=0.001)
        print(f"Found {len(context_docs)} relevant documents for query: {user_query}")
                
        # If no documents found with similarity search, use all documents
        if not context_docs or len(context_docs) == 0 or len(context_docs[0]) < 100:
            # Use all documents but limit total context size
            max_docs = min(5, len(vector_store.documents))  # Use up to 5 documents
            context_docs = vector_store.documents[:max_docs]
            print(f"Using {len(context_docs)} full documents as context for query: {user_query}")
    else:       
        # No documents available
        context_docs = ["No documents available in the knowledge base yet. Please upload PDF files."]
            
    # Extract file sources from context documents
    file_sources = []
    processed_docs = []
            
    for doc in context_docs:
        # Parse source file information from the document
        try:
            if doc.startswith("SOURCE_FILE:"):
                # Extract the source file name from the tag
                source_line = doc.split('\n')[0]  # Get the first line with source info
                file_id = source_line.replace("SOURCE_FILE:", "").strip()
                        
                # Add to list of sources if not already included
                if file_id not in file_sources:
                    file_sources.append(file_id)
                        
                # Remove the source tag line for the content
                processed_content = "\n".join(doc.split('\n')[1:])
                processed_docs.append(processed_content)
            else:
                # No source tag, use as is
                processed_docs.append(doc)
        except Exception as e:
            print(f"Error parsing document source: {e}")
            processed_docs.append(doc)  # Use original if any error
            
    # Convert file IDs to original filenames for better human readability
    readable_sources = []
    for file_id in file_sources:
        if file_id in file_information:
            # Use the original filename stored during upload
            readable_sources.append(file_information[file_id]['filename'])
        else:
            # For files with UUID prefixes, extract the original name
            if '_' in file_id and any(c for c in file_id if c == '-'):
                # This is likely a UUID prefixed filename pattern
                try:
                    # Extract the part after UUID
                    original_name = file_id.split('_', 1)[1] if '_' in file_id else file_id
                    readable_sources.append(original_name)
                except:
                    # Fallback to the ID if extraction fails
                    readable_sources.append(file_id)
            else:
                # Just use the filename directly if no UUID pattern
                readable_sources.append(file_id)
            
    # Log context found for debugging
    if processed_docs:
        print(f"Found {len(processed_docs)} relevant context documents from {len(file_sources)} files for query: {user_query}")
        print(f"Source files: {', '.join(readable_sources)}")
        for i, doc in enumerate(processed_docs):
            print(f"Context {i+1}: {doc[:200]}...")
                    
    context = "\n\n".join(processed_docs)
            
    # Add detailed file sources to the context for the API with file prioritization hints
    if readable_sources:
        # Group files by type to help with prioritization hints
        file_categories = {
            'diem_chuan': [],
            'hoc_phi': [],
            'nganh_hoc': [],
            'co_so': [],
            'tuyen_sinh': [],
            'other': []
        }
                
        # Categorize files for better searching
        for filename in readable_sources:
            filename_lower = filename.lower()
            if 'diem' in filename_lower or 'chuan' in filename_lower:
                file_categories['diem_chuan'].append(filename)
            elif 'hoc_phi' in filename_lower or 'phi' in filename_lower:
                file_categories['hoc_phi'].append(filename)
            elif 'nganh' in filename_lower or 'khoa' in filename_lower:
                file_categories['nganh_hoc'].append(filename)
            elif 'co_so' in filename_lower or 'vat_chat' in filename_lower:
                file_categories['co_so'].append(filename)
            elif 'tuyen_sinh' in filename_lower or 'tuyen' in filename_lower:
                file_categories['tuyen_sinh'].append(filename)
            else:
                file_categories['other'].append(filename)
                
        # Add metadata about files and prioritization hints
        source_info = "\n\n### SOURCE FILES AND PRIORITIZATION HINTS:\n"
        source_info += "THIS INFORMATION IS FOUND IN THE FOLLOWING FILES: " + ", ".join(readable_sources) + "\n"
                
        # Add hints about file categories to help API prioritize correctly
        if file_categories['diem_chuan']:
            source_info += "\nFILES ABOUT ĐIỂM CHUẨN (PRIORITY FOR QUESTIONS ABOUT ADMISSION SCORES): " + ", ".join(file_categories['diem_chuan'])
        if file_categories['hoc_phi']:
            source_info += "\nFILES ABOUT HỌC PHÍ (PRIORITY FOR QUESTIONS ABOUT TUITION FEES): " + ", ".join(file_categories['hoc_phi'])
        if file_categories['nganh_hoc']:
            source_info += "\nFILES ABOUT NGÀNH HỌC (PRIORITY FOR QUESTIONS ABOUT MAJORS/DEPARTMENTS): " + ", ".join(file_categories['nganh_hoc'])
        if file_categories['co_so']:
            source_info += "\nFILES ABOUT CƠ SỞ VẬT CHẤT (PRIORITY FOR QUESTIONS ABOUT FACILITIES): " + ", ".join(file_categories['co_so'])
        if file_categories['tuyen_sinh']:
            source_info += "\nFILES ABOUT TUYỂN SINH (PRIORITY FOR QUESTIONS ABOUT ADMISSIONS): " + ", ".join(file_categories['tuyen_sinh'])
        if file_categories['other']:
            source_info += "\nOTHER FILES: " + ", ".join(file_categories['other'])
                    
        context += source_info
            
    # Debug: Log length of context
    print(f"Context length: {len(context)} characters")
            
    # Use Orchestrator-workers model for better search and extraction
    # Run the async orchestrator in a synchronous environment
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    try:
        response = await orchestrate_response(user_query, processed_docs, file_sources)
    
    finally:
        loop.close()
                
    # Fallback to regular Gemini if orchestrator fails or returns None
    if not response or (isinstance(response, str) and "error" in response.lower()):
        print(f"Orchestrator failed, falling back to standard Gemini API")
        try:
            response = generate_response(user_query, context, chat_history=None)
        except Exception as gemini_error:
            print(f"Error with Gemini API: {gemini_error}")
            # Provide a fallback response when both methods fail
            response = """
            <div class="alert alert-warning">
                <h4>Không tìm thấy thông tin</h4>
                <p>Xin lỗi, tôi không tìm thấy thông tin liên quan đến câu hỏi của bạn trong cơ sở dữ liệu hiện có.</p>
                <p>Câu hỏi của bạn có thể nằm ngoài phạm vi thông tin tuyển sinh hoặc các tài liệu đã tải lên. 
                Vui lòng thử lại với một câu hỏi khác về thông tin tuyển sinh, hoặc liên hệ với phòng tuyển sinh để được hỗ trợ thêm.</p>
            </div>
            """
            
    # Clean the response HTML to ensure proper rendering
    response = clean_html_response(response)
    elapsed_time = round(time.time() - start_time, 2)

    return response, elapsed_time

In [31]:
user_query = "Điểm chuẩn ngành Công nghệ thông tin năm 2024 là bao nhiêu?"
context_docs = vector_store_transformer.similarity_search(user_query, k=5, threshold=0.001)
context_docs

['SOURCE_FILE:diem_chuan.pdf\nĐIỂMCHUẨNQUATỪNGNĂM\nTrongđàotạochínhquy, điểmchuẩncủacácngànhtại ĐạihọcMởTP. HCMquacác\nnămđượctổnghợpnhưsau:\nPhươngthứcxéttuyểnkỳthitốtnghiệpTHPT:\nNgành Quảntrịkinhdoanh:\nNăm 2024: 20. 75\nNăm 2023: 24. 00\nNăm 2022: 23. 30\nNăm 2021: 26. 40\nNăm 2020: 24. 70\nNgành QuảntrịkinhdoanhChấtlượngcao:\nNăm 2024: 20. 00\nNăm 2023: 22. 60\nNăm 2022: 20. 00\nNăm 2021: 26. 40\nNăm 2020: 21. 65\nNgànhMarketing:\nNăm 2024: 24. 50\nNăm 2023: 25. 25\nNăm 2022: 25. 25\nNăm 2021: 26. 95\nNăm 2020: 25. 35\nNgành Kinhdoanhquốctế:\nNăm 2024: 23. 75\nNăm 2023: 24. 90\nNăm 2022: 24. 70\nNăm 2021: 26. 45\nNăm 2020: 25. 05\nNgànhTàichính-Ngânhàng:\nNăm 2024: 23. 20\nNăm 2023: 23. 90\nNăm 2022: 23. 60\nNăm 2021: 25. 85\nNăm 2020: 24. 00\nNgànhTàichính-NgânhàngChấtlượngcao:\nNăm 2024: 18. 00\nNăm 2023: 22. 00\nNăm 2022: 20. 60\nNăm 2021: 25. 25\nNăm 2020: 18. 50\nNgànhKếtoán:\nNăm 2024: 21. 00\nNăm 2023: 23. 80\nNăm 2022: 23. 30\nNăm 2021: 25. 70\nNăm 2020: 24. 00\nNgànhKếtoá

In [30]:
file_sources = []
processed_docs = []
            
for doc in context_docs:
    # Parse source file information from the document
    try:
        if doc.startswith("SOURCE_FILE:"):
            # Extract the source file name from the tag
            source_line = doc.split('\n')[0]  # Get the first line with source info
            file_id = source_line.replace("SOURCE_FILE:", "").strip()
                    
            # Add to list of sources if not already included
            if file_id not in file_sources:
                file_sources.append(file_id)
                    
            # Remove the source tag line for the content
            processed_content = "\n".join(doc.split('\n')[1:])
            processed_docs.append(processed_content)
        else:
            # No source tag, use as is
            processed_docs.append(doc)
    except Exception as e:
        print(f"Error parsing document source: {e}")
        processed_docs.append(doc)  # Use original if any error
        
# Convert file IDs to original filenames for better human readability
readable_sources = []
for file_id in file_sources:
    if file_id in file_information:
        # Use the original filename stored during upload
        readable_sources.append(file_information[file_id]['filename'])
    else:
        # For files with UUID prefixes, extract the original name
        if '_' in file_id and any(c for c in file_id if c == '-'):
            # This is likely a UUID prefixed filename pattern
            try:
                # Extract the part after UUID
                original_name = file_id.split('_', 1)[1] if '_' in file_id else file_id
                readable_sources.append(original_name)
            except:
                # Fallback to the ID if extraction fails
                readable_sources.append(file_id)
        else:
            # Just use the filename directly if no UUID pattern
            readable_sources.append(file_id)
        
# Log context found for debugging
if processed_docs:
    print(f"Found {len(processed_docs)} relevant context documents from {len(file_sources)} files for query: {user_query}")
    print(f"Source files: {', '.join(readable_sources)}")
    for i, doc in enumerate(processed_docs):
        print(f"Context {i+1}: {doc[:200]}...")

loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)
try:
    response = await orchestrate_response(user_query, processed_docs, file_sources)

finally:
    loop.close()

# print(response)
print(clean_html_response(response))

Found 8 relevant context documents from 8 files for query: Điểm chuẩn ngành Công nghệ thông tin năm 2024 là bao nhiêu?
Source files: diem_chuan.pdf, hoc_phi_hoc_bong.pdf, thong_tin_tuyen_sinh_2024.pdf, thong_tin_tuyen_sinh_2025.pdf, ky_thi_vsat.pdf, co_so_vat_chat.pdf, thong_tin_nganh_hoc.pdf, OU_info.pdf
Context 1: ĐIỂMCHUẨNQUATỪNGNĂM
Trongđàotạochínhquy, điểmchuẩncủacácngànhtại ĐạihọcMởTP. HCMquacác
nămđượctổnghợpnhưsau:
PhươngthứcxéttuyểnkỳthitốtnghiệpTHPT:
Ngành Quảntrịkinhdoanh:
Năm 2024: 20. 75
Năm 2023: 24...
Context 2: HỌCPHÍDỰKIẾNCỦACHƯƠNGTRÌNHĐÀOTẠO
ĐHCHÍNHQUYÁPDỤNGKHÓA2025-NĂMHỌC2025-2026
I. Họcphí
Chươngtrìnhchuẩn
1. NhómngànhCôngnghệsinhhọc, Côngnghệthựcphẩm:
-Họcphídựkiến: 28. 500. 000đ/năm
2. NhómngànhCôngngh...
Context 3: THÔNGTINTUYỂNSINHĐẠIHỌCCHÍNHQUYNăm 2024
BỘGIÁODỤCVÀĐÀOTẠO
TRƯỜNGĐẠIHỌCMỞTHÀNHPHỐHỒMINH
Địachỉ: 97VõVănTần, Quận3, TP. HồChíMinh.
Điệnthoại: 1800585884, Website: https: //tuyensinh. ou. edu. vn
THÔNGTI...
Context 4: THÔNGTINTUYỂNSINHĐẠIHỌCCHÍNHQUYNăm 20

In [11]:
async def call_with_retry(async_func, retries=5, delay=2):
    for attempt in range(retries):
        try:
            return await async_func()
        except Exception as e:
            if "429" in str(e) or "Too Many Requests" in str(e):
                delay *= 2  # Tăng delay lên gấp đôi nếu bị rate limit
            if attempt < retries - 1:
                print(f"Attempt {attempt+1} failed: {e}. Retrying after {delay} seconds...")
                await asyncio.sleep(delay)
            else:
                print(f"All attempts failed for query. Error: {e}")
                return "Lỗi khi tạo câu trả lời", 0.0

In [20]:
output_file = "result_st.csv"
batch_size = 10

# Xác định số dòng đã xử lý
if os.path.exists(output_file):
    df_done = pd.read_csv(output_file)
    num_done = len(df_done)
else:
    df_done = pd.DataFrame()
    num_done = 0

# Chọn 10 dòng tiếp theo để xử lý
df_batch = df.iloc[num_done:num_done + batch_size].copy()
df_batch["generated_response"] = ""
df_batch["time_taken"] = 0.0

# Xử lý batch
for idx, row in df_batch.iterrows():
    response, time_taken = await call_with_retry(
        lambda: generate_responses(vector_store_transformer, row['query'])
    )
    df_batch.at[idx, "generated_response"] = response
    df_batch.at[idx, "time_taken"] = time_taken
    time.sleep(40)

# Ghi nối vào file
header = not os.path.exists(output_file)  # chỉ ghi header nếu file chưa tồn tại
df_batch.to_csv(output_file, mode='a', index=False, header=header)

Found 8 relevant documents for query: Trang web của Khoa Quản trị kinh doanh là gì?
Found 8 relevant context documents from 8 files for query: Trang web của Khoa Quản trị kinh doanh là gì?
Source files: OU_info.pdf, thong_tin_nganh_hoc.pdf, hoc_phi_hoc_bong.pdf, thong_tin_tuyen_sinh_2024.pdf, thong_tin_tuyen_sinh_2025.pdf, diem_chuan.pdf, co_so_vat_chat.pdf, ky_thi_vsat.pdf
Context 1: Mãtrường: MBS
NHỮNGLÝDOBẠNNÊNCHỌNĐẠIHỌCMỞTHÀNH
PHỐHỒCHÍMINH( HCMC-OU)
HCMC-OUđượcthànhlậpvàoNăm 1990vàchínhthứctrởthànhtrườngcônglập
vàoNăm 2006. Với30nămkinhnghiệmtronghoạtđộngđàotạo, Trườngđã
khôngn...
Context 2: THÔNGTINVỀKHOAVÀNGÀNHĐÀOTẠO
1. KhoaCôngnghệthôngtin
KhoaCôngNghệThôngTin( CNTT) đượcthànhlậptừtháng09/1990,
vàlàmộttrongnhững Khoađượcthànhlậpđầutiêncủatrường Đại
họcMởTP. HồChíMinh. Trảiquagần35nămxâ...
Context 3: HỌCPHÍDỰKIẾNCỦACHƯƠNGTRÌNHĐÀOTẠO
ĐHCHÍNHQUYÁPDỤNGKHÓA2025-NĂMHỌC2025-2026
I. Họcphí
Chươngtrìnhchuẩn
1. NhómngànhCôngnghệsinhhọc, Côngnghệthựcphẩm:
-Họcphídựkiến: 28. 500. 000đ/năm
2

In [2]:
df_eva = pd.read_csv('eva_data/tfidf_responses.csv', encoding='utf-8')
df_eva.head()

,query,expected_response,generated_response,time_taken
0,Mã trường là gì?,Trường Đại học Mở TP.Hồ Chí Minh có mã là MBS.,Mã trường Đại học Mở Thành phố Hồ Chí Minh Mã ...,3.79
1,Sinh viên ngành Công nghệ thông tin học ở cơ s...,Ngành Công nghệ thông tin được đào tạo tại cơ ...,Địa điểm học ngành Công nghệ thông tin tại Đại...,2.19
2,Ngành Công nghệ sinh học của trường Đại học Mở...,Ngành Công nghệ sinh học được đào tạo ở Cơ sở ...,Địa điểm đào tạo ngành Công nghệ Sinh học tại ...,5.68
3,Điểm chuẩn ngành Ngôn ngữ Nhật Chất lượng cao ...,Điểm chuẩn ngành Ngôn ngữ Nhật Chất lượng cao ...,Điểm chuẩn ngành Ngôn ngữ Nhật Chất lượng cao ...,3.12
4,Điểm chuẩn ngành Ngôn ngữ Hàn Quốc theo phương...,Điểm chuẩn ngành Ngôn ngữ Hàn Quốc theo phương...,Điểm chuẩn ngành Ngôn ngữ Hàn Quốc - Phương th...,3.99


In [86]:
df_eva.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   query               120 non-null    object 
 1   expected_response   120 non-null    object 
 2   generated_response  120 non-null    object 
 3   time_taken          120 non-null    float64
dtypes: float64(1), object(3)
memory usage: 3.9+ KB


In [3]:
import unicodedata
def clean_text(text):
    text = unicodedata.normalize("NFKC", str(text).lower())  # chuẩn hóa unicode
    text = re.sub(r'[^\w\s]', '', text)  # xóa dấu câu
    text = re.sub(r'\s+', ' ', text).strip()  # xóa khoảng trắng thừa
    return text.strip()

In [4]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from underthesea import word_tokenize

rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
smooth_fn = SmoothingFunction().method1

# Calculate ROUGE and BLEU scores for each row in df
rougeL_precision_list = []
rougeL_recall_list = []
rougeL_f1_list = []
bleu_list = []

for ref, gen in zip(df_eva['expected_response'], df_eva['generated_response']):
    # Calculate ROUGE-L score
    rouge_result = rouge.score(clean_text(ref), clean_text(gen))
    rougeL_precision_list.append(rouge_result['rougeL'].precision)
    rougeL_recall_list.append(rouge_result['rougeL'].recall)
    rougeL_f1_list.append(rouge_result['rougeL'].fmeasure)
    
    # Calculate BLEU score
    ref_tokens = word_tokenize(clean_text(ref), format="text").split()
    gen_tokens = word_tokenize(clean_text(gen), format="text").split()
    bleu_score = sentence_bleu([ref_tokens], gen_tokens, smoothing_function=smooth_fn)
    bleu_list.append(bleu_score)

# Add scores to the DataFrame
df_eva["rougeL_precision"] = rougeL_precision_list
df_eva["rougeL_recall"] = rougeL_recall_list
df_eva["rougeL_f1"] = rougeL_f1_list
df_eva['bleu_score'] = bleu_list

# Display the updated DataFrame
df_eva[['query', 'expected_response', 'generated_response', 'rougeL_precision', 'rougeL_recall', 'rougeL_f1', 'bleu_score']].head()

c:\Users\Win\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,query,expected_response,generated_response,rougeL_precision,rougeL_recall,rougeL_f1,bleu_score
0,Mã trường là gì?,Trường Đại học Mở TP.Hồ Chí Minh có mã là MBS.,Mã trường Đại học Mở Thành phố Hồ Chí Minh Mã ...,0.093023,0.923077,0.169014,0.018105
1,Sinh viên ngành Công nghệ thông tin học ở cơ s...,Ngành Công nghệ thông tin được đào tạo tại cơ ...,Địa điểm học ngành Công nghệ thông tin tại Đại...,0.120172,0.800000,0.208955,0.012037
2,Ngành Công nghệ sinh học của trường Đại học Mở...,Ngành Công nghệ sinh học được đào tạo ở Cơ sở ...,Địa điểm đào tạo ngành Công nghệ Sinh học tại ...,0.181518,0.714286,0.289474,0.062348
3,Điểm chuẩn ngành Ngôn ngữ Nhật Chất lượng cao ...,Điểm chuẩn ngành Ngôn ngữ Nhật Chất lượng cao ...,Điểm chuẩn ngành Ngôn ngữ Nhật Chất lượng cao ...,0.279070,0.960000,0.432432,0.140405
4,Điểm chuẩn ngành Ngôn ngữ Hàn Quốc theo phương...,Điểm chuẩn ngành Ngôn ngữ Hàn Quốc theo phương...,Điểm chuẩn ngành Ngôn ngữ Hàn Quốc - Phương th...,0.400000,1.000000,0.571429,0.298179


In [5]:
from sentence_transformers import SentenceTransformer, util

# Load model
model = SentenceTransformer("bkai-foundation-models/vietnamese-bi-encoder")

# Tính cosine similarity
similarities = []
for i, row in df_eva.iterrows():
    emb1 = model.encode(clean_text(row['expected_response']), convert_to_tensor=True)
    emb2 = model.encode(clean_text(row['generated_response']), convert_to_tensor=True)
    score = util.cos_sim(emb1, emb2).item()
    similarities.append(score)

# Thêm cột similarity
df_eva['similarity'] = similarities
df_eva[['query', 'expected_response', 'generated_response', 'similarity']].head()

,query,expected_response,generated_response,similarity
0,Mã trường là gì?,Trường Đại học Mở TP.Hồ Chí Minh có mã là MBS.,Mã trường Đại học Mở Thành phố Hồ Chí Minh Mã ...,0.622806
1,Sinh viên ngành Công nghệ thông tin học ở cơ s...,Ngành Công nghệ thông tin được đào tạo tại cơ ...,Địa điểm học ngành Công nghệ thông tin tại Đại...,0.486755
2,Ngành Công nghệ sinh học của trường Đại học Mở...,Ngành Công nghệ sinh học được đào tạo ở Cơ sở ...,Địa điểm đào tạo ngành Công nghệ Sinh học tại ...,0.699876
3,Điểm chuẩn ngành Ngôn ngữ Nhật Chất lượng cao ...,Điểm chuẩn ngành Ngôn ngữ Nhật Chất lượng cao ...,Điểm chuẩn ngành Ngôn ngữ Nhật Chất lượng cao ...,0.795045
4,Điểm chuẩn ngành Ngôn ngữ Hàn Quốc theo phương...,Điểm chuẩn ngành Ngôn ngữ Hàn Quốc theo phương...,Điểm chuẩn ngành Ngôn ngữ Hàn Quốc - Phương th...,0.881727


In [6]:
avg_similarity = df_eva['similarity'].mean()
print(f"Average similarity score: {avg_similarity:.4f}")

Average similarity score: 0.6051


In [90]:
from bert_score import score

# Lấy danh sách câu gốc và câu sinh
references = df_eva["expected_response"].astype(str).values.tolist()
candidates = df_eva["generated_response"].astype(str).values.tolist()

# Tính BERTScore
P, R, F1 = score(candidates, references, lang="vi") 

# Thêm kết quả vào DataFrame
df_eva["bertscore_precision"] = P.tolist()
df_eva["bertscore_recall"] = R.tolist()
df_eva["bertscore_f1"] = F1.tolist()

# Hiển thị kết quả
df_eva[['query', 'expected_response', 'generated_response', 'bertscore_precision','bertscore_recall','bertscore_f1']].head()

,query,expected_response,generated_response,bertscore_precision,bertscore_recall,bertscore_f1
0,Mã trường là gì?,Trường Đại học Mở TP.Hồ Chí Minh có mã là MBS.,"Thông tin chung Tôi xin lỗi, nhưng tôi không t...",0.494663,0.602544,0.543300
1,Sinh viên ngành Công nghệ thông tin học ở cơ s...,Ngành Công nghệ thông tin được đào tạo tại cơ ...,Địa điểm đào tạo ngành Công nghệ thông tin tại...,0.703445,0.862794,0.775014
2,Ngành Công nghệ sinh học của trường Đại học Mở...,Ngành Công nghệ sinh học được đào tạo ở Cơ sở ...,Ngành Công nghệ sinh học - Đại học Mở TP.HCM: ...,0.781573,0.758689,0.769961
3,Điểm chuẩn ngành Ngôn ngữ Nhật Chất lượng cao ...,Điểm chuẩn ngành Ngôn ngữ Nhật Chất lượng cao ...,Điểm Chuẩn Ngành Ngôn ngữ Nhật Chất lượng cao ...,0.735787,0.823686,0.777259
4,Điểm chuẩn ngành Ngôn ngữ Hàn Quốc theo phương...,Điểm chuẩn ngành Ngôn ngữ Hàn Quốc theo phương...,Điểm Chuẩn Ngành Ngôn Ngữ Hàn Quốc - Phương Th...,0.721483,0.824475,0.769548


In [91]:
df_eva.describe()

,time_taken,rougeL_precision,rougeL_recall,rougeL_f1,bleu_score,similarity,bertscore_precision,bertscore_recall,bertscore_f1
count,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000
mean,8.158750,0.232854,0.819576,0.331467,0.108250,0.695056,0.682824,0.788022,0.730216
std,3.462559,0.150888,0.192249,0.164817,0.121087,0.142487,0.083895,0.079464,0.076612
min,3.480000,0.000000,0.000000,0.000000,0.000000,0.202860,0.490263,0.531360,0.511419
25%,5.840000,0.135741,0.756466,0.225494,0.019425,0.608692,0.628333,0.748174,0.679588
50%,7.260000,0.206857,0.875000,0.321463,0.071758,0.699888,0.682985,0.804697,0.730563
75%,8.962500,0.291744,0.958333,0.422549,0.139278,0.803403,0.739986,0.845354,0.782217
max,20.490000,0.945946,1.000000,0.927152,0.647146,0.963653,0.918759,0.940345,0.929427


In [72]:
print("Average time taken: ", round(df_eva['time_taken'].mean(),2))

Average time taken:  8.16


In [ ]:
df_eva.to_csv('eva_data/eva_res_tfidf.csv', index=False, encoding='utf-8-sig')

In [92]:
df_eva.to_csv('eva_data/eva_res_st.csv', index=False, encoding='utf-8-sig')

In [3]:
df_grok = pd.read_csv('eva_data/st_grok_comparison.csv', encoding='utf-8')
df_grok.head()

,query,expected_response,generated_response,key_terms,results
0,Mã trường là gì?,Trường Đại học Mở TP.Hồ Chí Minh có mã là MBS.,"Thông tin chung Tôi xin lỗi, nhưng tôi không t...",MBS,0
1,Sinh viên ngành Công nghệ thông tin học ở cơ s...,Ngành Công nghệ thông tin được đào tạo tại cơ ...,Địa điểm đào tạo ngành Công nghệ thông tin tại...,"Nhà Bè, Khu Dân cư Nhơn Đức, Huyện Nhà Bè",1
2,Ngành Công nghệ sinh học của trường Đại học Mở...,Ngành Công nghệ sinh học được đào tạo ở Cơ sở ...,Ngành Công nghệ sinh học - Đại học Mở TP.HCM: ...,"Nhà Bè, Bình Dương, 68, đường Lê Thị Trung, Th...",1
3,Điểm chuẩn ngành Ngôn ngữ Nhật Chất lượng cao ...,Điểm chuẩn ngành Ngôn ngữ Nhật Chất lượng cao ...,Điểm Chuẩn Ngành Ngôn ngữ Nhật Chất lượng cao ...,700,1
4,Điểm chuẩn ngành Ngôn ngữ Hàn Quốc theo phương...,Điểm chuẩn ngành Ngôn ngữ Hàn Quốc theo phương...,Điểm Chuẩn Ngành Ngôn Ngữ Hàn Quốc - Phương Th...,730,1


In [4]:
df_grok.groupby('results')['query'].count()

results
0    27
1    93
Name: query, dtype: int64

In [94]:
percent_correct = (df_grok['results'].sum() / len(df_grok)) * 100
print(f"Phần trăm câu trả lời đúng là: {percent_correct:.2f}%")

Phần trăm câu trả lời đúng là: 77.50%


In [5]:
df_grok1 = pd.read_csv('eva_data/tfidf_grok_comparison.csv', encoding='utf-8')
df_grok1.head()

,query,expected_response,generated_response,key_terms,results
0,Mã trường là gì?,Trường Đại học Mở TP.Hồ Chí Minh có mã là MBS.,Mã trường Đại học Mở Thành phố Hồ Chí Minh Mã ...,MBS,1
1,Sinh viên ngành Công nghệ thông tin học ở cơ s...,Ngành Công nghệ thông tin được đào tạo tại cơ ...,Địa điểm học ngành Công nghệ thông tin tại Đại...,"Nhà Bè, Khu Dân cư Nhơn Đức, Huyện Nhà Bè",0
2,Ngành Công nghệ sinh học của trường Đại học Mở...,Ngành Công nghệ sinh học được đào tạo ở Cơ sở ...,Địa điểm đào tạo ngành Công nghệ Sinh học tại ...,"Nhà Bè, Bình Dương, 68, đường Lê Thị Trung, Th...",1
3,Điểm chuẩn ngành Ngôn ngữ Nhật Chất lượng cao ...,Điểm chuẩn ngành Ngôn ngữ Nhật Chất lượng cao ...,Điểm chuẩn ngành Ngôn ngữ Nhật Chất lượng cao ...,700,1
4,Điểm chuẩn ngành Ngôn ngữ Hàn Quốc theo phương...,Điểm chuẩn ngành Ngôn ngữ Hàn Quốc theo phương...,Điểm chuẩn ngành Ngôn ngữ Hàn Quốc - Phương th...,730,1


In [8]:
df_grok1.groupby('results')['query'].count()

results
0    38
1    82
Name: query, dtype: int64

In [7]:
percent_correct = (df_grok1['results'].sum() / len(df_grok1)) * 100
print(f"Phần trăm câu trả lời đúng là: {percent_correct:.2f}%")

Phần trăm câu trả lời đúng là: 68.33%
